## Problem statement

The Bank Churn prediction problem statement is about building a neural network-based classifier that can determine whether a bank customer will leave or not in the next 6 months. 
- The problem of 'Churn is when customer will leave and join another service provider. 
- The dataset has customer info such as age, gender, demographics , and transactions with the bank.
- The bank wants to identify customers likely to churn balances below the minimum balance.

**Objective**:
Given a Bank customer, build a neural network-based classifier that can determine whether they will leave or not in the next 6 months.

**Context**:
Businesses like banks that provide service have to worry about the problem of 'Churn' i.e. customers leaving and joining another service provider. It is important to understand which aspects of the service influence a customer's decision in this regard. Management can concentrate efforts on the improvement of service, keeping in mind these priorities.

**Data Description**:
The case study is from an open-source dataset from Kaggle. The dataset contains 10,000 sample points with 14 distinct features such as CustomerId, CreditScore, Geography, Gender, Age, Tenure, Balance, etc. Link to the Kaggle project site: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling

**Data Dictionary:**

RowNumber: Row number.

CustomerId: Unique identification key for different customers.

Surname: Surname of the customer

Credit Score: Credit score is a measure of an individual's ability to pay back the borrowed amount. It is the numerical representation of their creditworthiness. A credit score is a 3-digit number that falls in the range of 300-900, 900 being the highest.

Geography: The country to which the customer belongs.

Gender: The gender of the customer.

Age: Age of the customer.

Tenure: The period of time a customer has been associated with the bank.

Balance: The account balance (the amount of money deposited in the bank account) of the customer.

NumOfProducts: How many accounts, bank account affiliated products the person has.

HasCrCard: Does the customer have a credit card through the bank?

IsActiveMember: Subjective, but for the concept

EstimatedSalary: Estimated salary of the customer.

Exited: Did they leave the bank after all?

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
# pickle is used for pickling beacuse when we use standard scaler and lable encoder, so it is 
# important to pickle this file so we can reuse it during deployment

In [39]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [40]:
## Preprocess the data
### Drop irrelevant columns 
# (here rownumber,customerid, surname)these 3 features are not that much important
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) #axis=1 means column wise drop
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [41]:
## Encode Categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

# female-0 male-1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


For Encoding - geography column
if we directly apply OHE then we'll get France-0, Spain-1, Germany-2
when these type of no. coming up then it will create problem beacuse at the end ann is all about numerical calculation since the label of germany is 2 , it will consider germany is greater than spain or spain is greater than France , so this should not happen.

so for this particular case , we will mot directly convert this with help of label encoder but instead we'll go ahead right some code and then use OHE(it will give values of 0's and 1's)


In [42]:
## Onehot encode 'Geography' column

from sklearn.preprocessing import OneHotEncoder
# onehot_encoder_geo= OneHotEncoder(sparse=False)
onehot_encoder_geo= OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [43]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [44]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

we'll convert geo_encoder sparse array into dataframe i.e geo_encoded_df using colm name from get_feature_name_out

In [45]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [46]:
## Combine one hot encoder columns with the original data
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [47]:
## Save the encoders and scaler

#open pickle file in write byte mode because this is deserialized or serialized file itself, so we'll be writing in the form of bytes..this as file(temporary var)
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)    

In [48]:
## Divide the dataset into independent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [49]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [50]:
# save this scaler file in pickel format
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [51]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


Now our data is ready & now we are going to train ANN

# ANN Implementation

In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [53]:
(X_train.shape[1],)
#means its a single dimension and it has 12 inputs

(12,)

In [54]:
## Build our ANN Model
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid') ## output layer
]

)

In [55]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()
loss

In [57]:
## compile the model (for forward and backward propagation)
# model.compile(optimizer="adam",loss="binary_crossentropy",metric=['accuracy'])

# there are two ways of compilation of model with optimizer and loss (we can directly use this particular keyword or initialize)
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [58]:
## Set up the Tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [59]:
## Set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [60]:
## Train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 3s 7ms/step - loss: 0.3957 - accuracy: 0.8310 - val_loss: 0.3602 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3542 - accuracy: 0.8518 - val_loss: 0.3510 - val_accuracy: 0.8520
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3454 - accuracy: 0.8564 - val_loss: 0.3471 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3455 - accuracy: 0.8584 - val_loss: 0.3448 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3437 - accuracy: 0.8575 - val_loss: 0.3377 - val_accuracy: 0.8560
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3371 - accuracy: 0.8610 - val_loss: 0.3503 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3369 - accuracy: 0.8635 - val_loss: 0.3420 - val_accuracy: 0.8545

In [61]:
model.save('model.h5') # h5 file will be compatible with keras

e:\UDemy Final\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [62]:
## Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [63]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 6684), started 0:12:48 ago. (Use '!kill 6684' to kill it.)

In [ ]:
## Load the pickle file

# vedio-67 Pediction with trained ANN model
# start with new file "prediction.ipynb"
